# Package: pandas
<img src = "https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/03/pandas.jpg" width = 200></img>
* We will focus on **DataFrame**.
* There are useful functions built in DataFrame:
    * read_csv(), read_excel()
    * sort_index(), sort_values(), dropna(), fillna()
    * head(), tail(), info(), shape, count()
    * index, columns, values
    * plot.line(), hist()
    * loc(), iloc()
    * mean(), std(), max(), min(), describe(), pct_change()
    * rolling(), apply()
    * pd.concat(), pd.merge()
* We start with the market data of 2330 and accomplish the following tasks:
    * Reading data from excel files;
    * Preprocessing data: dealing with the index;
    * Making plots for the data;
    * Making a birdview of the table;
    * Accessing data;
    * Calculating some indicators;
    * Output to files.

In [ ]:
!wget https://www.csie.ntu.edu.tw/~d00922011/python/data/2330tw.xlsx

In [ ]:
import pandas as pd

df = pd.read_excel("2330tw.xlsx")
df.head()

## Preprocessing: Index

In [ ]:
df.sort_values(by = "Date", inplace = True, ascending = True) # df = df.sort_values(by = "Date", ascending = True)

df.head()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"]) # convert date strings to datetime objects
df.set_index("Date", inplace = True)

In [ ]:
df.head() # show the first 5 items

## Birdview of Table

In [ ]:
df.info()

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
df.describe()

## Data Access


### Column Selection by Name

In [ ]:
df["Close"]

### Row Selection by Index Label

In [ ]:
df.loc["2021-02-25"]

In [ ]:
df.loc["2021-02-01" : "2021-02-28"]

### Data Selection by Index Location: iloc()

In [ ]:
df.iloc[-5:] # last 5 trading days

In [ ]:
df.index[-1]

## Plotting

In [ ]:
df["Close"].plot.line(grid = True)

In [ ]:
ax = df[["Open", "High", "Low", "Close"]][-100:].plot.line(grid = True)
ax.set_ylabel("Price")

In [ ]:
?df.plot

In [ ]:
df["Volume"][-100:].plot.bar() # See https://stackoverflow.com/questions/30133280/pandas-bar-plot-changes-date-format

### Calculation: rolling(), mean()

In [ ]:
df["SMA5"] = df["Close"].rolling(5).mean() # SMA: simple moving average

In [ ]:
df[["Close", "SMA5"]].plot.line(grid = True)

#### Exercise: Bias

In [ ]:
df["BIAS"] = (df["Close"] - df["SMA5"]) / df["SMA5"] * 100
ax = df["BIAS"].plot.line(grid = True)
ax.set_ylabel("Bias (%)")

### Calculation: Percentage Change

In [ ]:
df["Return rate (%)"] = df["Close"].pct_change() * 100

In [ ]:
df["Return rate (%)"].plot.line(grid = True)

In [ ]:
ax = df["Return rate (%)"].plot.hist(bins = 30, grid = True)
ax.set_xlabel("Return rate (%)")

### Output to Files

In [ ]:
df.to_excel("output.xlsx") # output to a excel file

## More Examples: Data Acquisition with Pandas

### Example 1: 三大法人買賣日報表

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime
from io import StringIO

def crawler(date):
    
    r = requests.get('http://www.tse.com.tw/fund/T86?response=csv&date={0}&selectType=ALLBUT0999'.format(date))
    df = pd.read_csv(StringIO(r.text), header = 1).dropna(how = 'all', axis = 1).dropna(how = 'any')
    
    df = df.astype(str).apply(lambda s: s.str.replace(',', ''))
    df['stock_id'] = df['證券代號'].str.replace('=', '').str.replace('"', '')
    df = df.set_index('stock_id')
    
    return df.apply(lambda s: pd.to_numeric(s, errors = 'coerce')).dropna(how = 'all', axis = 1)

In [ ]:
df = crawler("20210528")

df.head(10)

In [ ]:
df.loc["2330"]

### Example 2: Pandas Datareader
* Official website: https://pandas-datareader.readthedocs.io/en/latest/

In [ ]:
!pip install pandas_datareader

* Free to register for API KEY: https://www.alphavantage.co/
* Note that only 500 requests per day.

In [ ]:
import os
import pandas_datareader.data as web
from datetime import datetime

f = web.DataReader("AAPL", "av-daily", 
          start = datetime(2020, 1, 1),
           end = datetime(2021, 4, 20),
         api_key = "0CW0TVWZO87J25H2")
f

### Example 3: Real-Time Foreign Exchange from Alpha Vantage

In [ ]:
import os
import pandas_datareader.data as web
from datetime import datetime

fx = web.DataReader(["USD/JPY", "BTC/USD"], "av-forex", api_key = "0CW0TVWZO87J25H2")
print(fx)